In [ ]:
'''
@auther:  Harsh Pal
@github: github.com/harshpaal

'''

In [ ]:

!pip install tesseract-ocr
!pip install tesseract
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import imutils
import tesseract

from google.colab.patches import cv2_imshow

In [25]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'

In [16]:
!pip install Pillow
!pip install pytesseract
import pytesseract

In [ ]:
!sudo apt install tesseract-ocr

#### Function to crop the image

In [2]:
def get_top_down_view(img, rect):
    #Get All corner points of rect
#     rect = np.zeros((4, 2), np.float32)
    
#     points_sum = points.sum(axis=1)
#     points_diff = np.diff(points, axis=1)
    
#     rect[0] = points[np.argmin(points_sum)]
#     rect[2] = points[np.argmax(points_sum)]
    
#     rect[1] = points[np.argmin(points_diff)]
#     rect[3] = points[np.argmax(points_diff)]
    
    top_left, top_right = rect[0], rect[1]
    bottom_left, bottom_right = rect[3], rect[2]
    
    width = max(
        int(np.sqrt(((top_left[0] - top_right[0]) ** 2) + ((top_left[1] - top_right[1])**2))),
        int(np.sqrt(((bottom_left[0] - bottom_right[0]) ** 2) + ((bottom_left[1] - bottom_right[1]) ** 2)))
    )
    
    height = max(
        int(np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))),
        int(np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + ((top_right[1] - bottom_right[1]) ** 2)))
    )
    
    mat = np.array([
        [0, 0],
        [width-1, 0],
        [width-1, height-1],
        [0, height-1]
    ], np.float32)
    
    M = cv2.getPerspectiveTransform(rect, mat)
    changed_img = cv2.warpPerspective(img, M, (width, height))
    
    return changed_img

In [3]:
def get_cropped_image(img_1):
    resized_ratio = img_1.shape[1] / 500
    img = imutils.resize(img_1, width=500)

    #convert to grey scale
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_grey = cv2.GaussianBlur(img_grey, (5, 5), 0)

    #Detect edges from image
    img_edge = cv2.Canny(img_grey, 75, 150)
    
    contours = cv2.findContours(img_edge.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    #contours = sorted(contours, key = cv2.contourArea, reverse=True)[:5]

    #Iterate over contours
    (x, y, w, h) = cv2.boundingRect(contours[0])

    max_area = w * h
    cnt = contours[0]
    
    for c in contours:
#     perimeter = cv2.arcLength(c, True)
    
#     approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
#     print(len(approx))
    
#     if len(approx) == 4:
#         target = approx
#         break

        (x_t, y_t, w_t, h_t) = cv2.boundingRect(c)

        if (w_t * h_t) > max_area:
            max_area = w_t * h_t
            x, y, w, h = x_t, y_t, w_t, h_t
            cnt = c
            
    boundary = np.array([
    [x, y], 
    [x+w, y],
    [x+w, y+h],
    [x, y+h]
    ], np.float32)
    
    boundary = boundary * resized_ratio
    
    top_view = get_top_down_view(img_1, boundary)
    top_view = imutils.resize(top_view, width=500)

    return top_view

In [18]:
def get_text(img):
    text = pytesseract.image_to_string(img, lang='eng')
    return text

In [5]:
def get_text_from_image(image):
    
    print('Start detecting contours...')
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    
    gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dst = cv2.GaussianBlur(gray,(5,5),cv2.BORDER_DEFAULT)
    blur = cv2.medianBlur(gray,5)
    blur = cv2.morphologyEx(blur, cv2.MORPH_OPEN, sqKernel)

    ret3, thresh3 = cv2.threshold(gray,100,255, cv2.THRESH_BINARY)
    ret, thresh = cv2.threshold(gray,100,255, cv2.THRESH_BINARY_INV)
    
    tophat = cv2.morphologyEx(thresh, cv2.MORPH_TOPHAT, rectKernel)
    gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
    gradX = gradX.astype("uint8")
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
    thresh2 = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    thresh2 = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, sqKernel)
    
    cnts = cv2.findContours(thresh2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    print('Contour detection ended....')
    locs = []
    
    for c in cnts[0]:
        # compute the bounding box of the contour, then use the
        # bounding box coordinates to derive the aspect rati
        (x, y, w, h) = cv2.boundingRect(c)
        if h == 0:
            h = 1
        ar = float(w) / h

        #print(x, y, w, h)
        #locs.append((x, y, w, h))

        # since credit cards used a fixed size fonts with 4 groups
        # of 4 digits, we can prune potential contours based on the
        # aspect ratio
    #     if ar > 0 and ar < 24:
    #         #cv2.rectangle(image, (x,y),(x+w,y+h),(255,0,0),1)
    #         #locs.append((x, y, w, h))
    #         # contours can further be pruned on minimum/maximum width
    #         # and height
    #         if (w > 0 and w < 500) and (h > 5 and h < 19):
    #         # append the bounding box region of the digits group
    #         # to our locations list
        locs.append((x, y, w, h))
        #cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 1)
        
    print(len(locs), 'contours found in image...')
    
    print('Starting recognition of text......')
    
    text_from_image = []
    for loc in locs:
    #cv2.rectangle(image, (loc[0], loc[1]), (loc[0]+loc[2], loc[1]+loc[3]), (0, 255, 0), 1)
        part = image[max(0, loc[1]-2):min(image.shape[0], loc[1]+loc[3]+2), max(0, loc[0]-2):min(image.shape[1],loc[0]+loc[2]+2)]
        text = get_text(part)
        if len(text) > 0:
            text_from_image.append(text)
            
    print('Text recognition ended....')
            
    return text_from_image

In [6]:
def get_face_from_image(frame):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,125), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        return roi_color

In [ ]:
img = cv2.imread('img2.jpeg')
img_1 = imutils.resize(img, width=500)
cv2_imshow(img_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

top_view = get_cropped_image(img)
cv2_imshow(top_view)
cv2.waitKey(0)
cv2.destroyAllWindows()

text_from_image = get_text_from_image(top_view)

In [ ]:
print(text_from_image)

In [ ]:
face = get_face_from_image('img2.jpeg')
cv2.imshow('Face', face)
cv2.waitKey(0)
cv2.destroyAllWindows()